In [1]:
import pandas as pd
from gurobipy import *

In [2]:
with open('test_case.txt') as text_file:
    data = text_file.read().splitlines()

data

['Test Sequence with length 20:',
 'CGUCUUCACUACAGCAUCGG',
 '',
 'Test Sequence with length 30:',
 'GACCUUACUGGGUACGAUUUACUGGAGGAC',
 '',
 'Test Sequence with length 50:',
 'GGCCAGACUGGUGGUGUGACUCCAGGCUAACCGGAUACGCGUGCCUCGGG',
 '',
 'Test Sequence with length 100:',
 'UAUGCAGGUCGCGUUUUUCCACUGCCUAGAUAGCUCUGAGGGUACACUUAGUUCAGCACAUAAGAGGGAUCAUACUAGGUCCGCGUCUUACCUCCUACGA',
 '',
 'Test Sequence with length 200:',
 'CCAUGUGGACGAGUUCACGUUGCGGUCUGCCGCGAAGAAUGAGUCGUCCAAUUUUUUUUUCCUGUAUGACGGAUAGAGUGUAGGGUGGGUGAUAGUGUAACUCCGUCCCGGCGUUGUCUAUGGCAGUCUAAUUGUAGUGAGCCAAAAUAUUUCAGAGAGUAACCAUUACAGGCUACCACACGACCCUGAUCCUAUUGUAU',
 '']

In [3]:
s = data[1]
len(s)

20

In [4]:
m = Model("Simple")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-12


In [5]:
m.setParam('OutputFlag',True)
m.setParam('TimeLimit', 5*60)

Set parameter TimeLimit to value 300


# Pre-Processing Data

In [6]:
ONE_N = list(range(1,len(s)+1))
ONE_N

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

## A_i, U_i, C_i, G_i

In [7]:
a = {}
u = {}
c = {}
g = {}

In [8]:
index = 1
for n in s:
    if (n == 'A'):
        a[index] = 1
        u[index] = 0
        c[index] = 0
        g[index] = 0
    if (n == 'U'):
        a[index] = 0
        u[index] = 1
        c[index] = 0
        g[index] = 0
    if (n == 'C'):
        a[index] = 0
        u[index] = 0
        c[index] = 1
        g[index] = 0
    if (n == 'G'):
        a[index] = 0
        u[index] = 0
        c[index] = 0
        g[index] = 1
    
    index += 1

In [9]:
print(a)
print(u)
print(c)
print(g)
print(s)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0}
{1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0}
{1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0}
{1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1}
CGUCUUCACUACAGCAUCGG


## X_ij

In [10]:
ls_tup = []
for i in range(1,len(s)):
    for j in range(i+1,len(s)+1):
        ls_tup.append((i,j))
        
x = m.addVars(ls_tup,name='X_ij',vtype=GRB.BINARY)
x

{(1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (

## Complementary Pairs

In [11]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if (a[i] + a[j] + c[i] + c[j] == 2):
            x[i,j] = 0
        if (a[i] + a[j] + g[i] + g[j] == 2):
            x[i,j] = 0
        if (u[i] + u[j] + c[i] + c[j] == 2):
            x[i,j] = 0
        if (u[i] + u[j] + g[i] + g[j] == 2):
            x[i,j] = 0

# Objective Function

In [12]:
m.setObjective((quicksum(x[i,j] for i in ONE_N for j in list(range(i+1,len(s)+1)))),GRB.MAXIMIZE)

# Constraints

## Character can be in at most 1 pair

In [13]:
m.addConstrs(quicksum([x[i,k] for i in range(1,k)]) + quicksum([x[k,j] for j in list(range(k+1,len(s)+1))]) <= 1 for k in ONE_N)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>}

## Non-Crossing Constraint

In [14]:
m.addConstrs(quicksum(x[k,l] for k in range(i+1,j) for l in [i for i in list(range(1,len(s)+1)) if i >= j]) <= (10**6)*(1-x[i,j]) for i in range(1,len(s)+1) for j in range(i+1,len(s)+1))

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model

In [15]:
# Optimize the model
m.optimize()
# Print the result
status_code = {1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD', 5: 'UNBOUNDED'}
status = m.status
print("The optimization status is {}".format(status_code[status]))
if status == 2:
# Retrieve variables value
    print("Optimal solution:")
    for v in m.getVars():
        if (v.x == 1):
            print("%s = %g" % (v.varName, v.x))
    print("Optimal objective value:\n{}".format(m.objVal))
    print("Runtime is:", m.Runtime)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 210 rows, 190 columns and 1743 nonzeros
Model fingerprint: 0x1edcdabb
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 6.0000000
Presolve removed 179 rows and 161 columns
Presolve time: 0.01s
Presolved: 31 rows, 29 columns, 175 nonzeros
Found heuristic solution: objective 7.0000000
Variable types: 0 continuous, 29 integer (29 binary)

Root relaxation: objective 8.000000e+00, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.0000000    8.00000  0.00%  

In [16]:
m.write("check.lp")